In [1]:
library("mxnet")
sales_data <- read.table("data/shampoo_sales.txt",sep = ",",header = TRUE)

# We require only one column from the dataset
sales_data <- as.data.frame(sales_data[,2])

ERROR: Error in library("mxnet"): there is no package called 'mxnet'


In [ ]:
min_max_scaler <- function(x) {
  (x - min(x))/(max(x) - min(x))
}

norm_sales_data <- min_max_scaler(sales_data)
t_sales_data <- t(norm_sales_data)

In [ ]:
n_dim <- 1
seq_len <- 4
num_samples <- 7

# extract only required data from dataset
x_data <- t_sales_data[1, 1:(seq_len * num_samples)]
dim(x_data) <- c(n_dim, seq_len, num_samples)

y_data <- t_sales_data[1, 2:(1+(seq_len * num_samples))]
dim(y_data) <- c(seq_len, num_samples)

In [ ]:
batch_size <- 3
train_ids <- 1:4
val_ids <- 5:6

## create data iterators
train_data <- mx.io.arrayiter(data = x_data[,,train_ids, drop = F],label = y_data[, train_ids], batch.size = batch_size,shuffle = TRUE)
val_data <- mx.io.arrayiter(data = x_data[,,val_ids, drop = F], label = y_data[, val_ids], batch.size = batch_size, shuffle = FALSE)

In [ ]:
symbol <- rnn.graph(num_rnn_layer = 2,
                    num_hidden = 30,
                    input_size = NULL,
                    num_embed = NULL,
                    num_decode = 1,
                    masking = F, 
                    loss_output = "linear",
                    ignore_label = -1, 
                    cell_type = "lstm", 
                    output_last_state = T,
                    config = "one-to-one")

In [ ]:
seq_metric_mse <- mx.metric.custom("MSE", function(label, pred) {
  label = mx.nd.reshape(label, shape = -1)
  pred = mx.nd.reshape(pred, shape = -1)
  res <- mx.nd.mean(mx.nd.square(label - pred))
  return(as.array(res))
})

In [ ]:
ctx <- mx.cpu()
initializer <- mx.init.Xavier(rnd_type = "gaussian",
                              factor_type = "avg", 
                              magnitude = 1)
optimizer <- mx.opt.create("adadelta",
                           rho = 0.9, 
                           eps = 1e-06, 
                           wd = 1e-06, 
                           clip_gradient = 1, 
                           rescale.grad = 1/batch_size)

In [ ]:
model <- mx.model.buckets(symbol = symbol, 
                                      train.data = train_data, 
                                      eval.data = val_data,
                                      num.round = 50, 
                                      ctx = ctx, 
                                      verbose = TRUE, 
                                      metric = seq_metric_mse, 
                                      initializer = initializer,
                                      optimizer = optimizer)

In [ ]:
internals <- model$symbol$get.internals()
sym_state <- internals$get.output(which(internals$outputs %in% "RNN_state"))
sym_state_cell <- internals$get.output(which(internals$outputs %in% "RNN_state_cell"))
sym_output <- internals$get.output(which(internals$outputs %in% "loss_output"))
symbol <- mx.symbol.Group(sym_output, sym_state, sym_state_cell)

In [ ]:
data <- mx.nd.array(x_data[, , 6, drop = F])
label <- mx.nd.array(y_data[, 6, drop = F])

inference_data <- mx.io.arrayiter(data = data, 
                              label = label, 
                              batch.size = 1, 
                              shuffle = FALSE)
infer <- mx.infer.rnn.one(infer.data = inference_data, 
                          symbol = symbol, 
                          arg.params = model$arg.params,
                          aux.params = model$aux.params, 
                          input.params = NULL, 
                          ctx = ctx)

In [ ]:
pred_length <- 3
predicted <- numeric()

In [5]:
for (i in 1:pred_length) {
  data <- mx.nd.array(x_data[, i, 7, drop = F])
  label <- mx.nd.array(y_data[i, 7, drop = F])
  infer.data <- mx.io.arrayiter(data = data, 
                                label = label, 
                                batch.size = 1, 
                                shuffle = FALSE)
  ## use previous RNN state values
  infer <- mx.infer.rnn.one(infer.data = infer.data,
                            symbol = symbol,
                            ctx = ctx, 
                            arg.params = model$arg.params,
                            aux.params = model$aux.params, 
                            input.params = 
                              list(rnn.state=infer[[2]], 
                                   rnn.state.cell = infer[[3]]))
  pred <- infer[[1]]
  predicted <- c(predicted, as.numeric(as.array(pred)))
}

predicted

ERROR: Error in eval(expr, envir, enclos): object 'pred_length' not found
